In [1]:
import pickle
import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from keras.layers import *
from keras import optimizers
from keras import backend as K
from sklearn.metrics import f1_score

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train_x,test_x = [],[]
for feat in sorted(glob.glob('../feature/bagging/*')):
    print('file path',feat)
    a,b,c = pickle.load(open(feat,'rb'))
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(c)
# load y
train = pd.read_csv("../input/new_data/train_set.csv")
train_x = np.nan_to_num(np.hstack(train_x))
test_x = np.nan_to_num(np.hstack(test_x))
print(train_x.shape)

file path ../feature/bagging/rnn_model_bagging1.pkl
(56590, 19) (56590, 19)
file path ../feature/bagging/rnn_model_bagging2.pkl
(56590, 19) (56590, 19)
file path ../feature/bagging/rnn_model_bagging3.pkl
(56590, 19) (56590, 19)
(56590, 57)


In [15]:
train_y=b
train_y = np.argmax(train_y,axis=1)

In [4]:
def self_f1_score(data, y_hat):
    y_true = data
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return f1_score(y_true, y_hat, average='macro')

In [17]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
def simple_ens(model_name,k=3,rnd=233,lr=0.05,feature_fraction=0.9,bagging_fraction=0.9,
               bag_frec=3,met='binary_logloss'):
#     kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=rnd)
    kf = KFold(n_splits=10, shuffle=False, random_state=233*rnd)
    test_pred, train_pred = np.zeros((102277,19)), np.zeros((102277,19))
    now_nfold=0
    for train_index, test_index in kf.split(train_x):
        
        now_nfold+=1
        print ("now is {} fold".format(now_nfold))
        curr_x,curr_y = train_x[train_index],train_y[train_index]
        hold_out_x,hold_out_y = train_x[test_index],train_y[test_index]
        d_test = test_x
        
        model = LogisticRegression(C=4, dual=True)
        model.fit(curr_x, curr_y)
        vaild = model.predict_proba(hold_out_x)
        train_pred[test_index] = vaild
        val_result = np.argmax(vaild,axis=1)
        print ('score is {}'.format(f1_score(hold_out_y, val_result, average='macro')))
        curr_test_pred = model.predict_proba(d_test)
        test_pred += curr_test_pred

    test_pred = test_pred / 10
    print('-------------------------------')
    return train_pred, test_pred

print('done')

done


In [18]:
train_pred, lr_res = simple_ens('lgb',10,233,0.05,0.6)

now is 1 fold
score is 0.8039810270702624
now is 2 fold
score is 0.8030457387930755
now is 3 fold
score is 0.8146128808873921
now is 4 fold
score is 0.8054256702430308
now is 5 fold
score is 0.7948980959271739
now is 6 fold
score is 0.8091409654932807
now is 7 fold
score is 0.7980826811421065
now is 8 fold
score is 0.800144809811262
now is 9 fold
score is 0.8034801150736163
now is 10 fold
score is 0.8009246980641775
-------------------------------


In [21]:
name = ["class_prob_%s"%i for i in range(1,20)]
df_lgb = pd.DataFrame(lr_res, columns=name)
# df_lgb = df_lgb.drop('class_prob_20', axis=1)
df_lgb.to_csv('../pro/stacking_lr_bagging.csv', index=False)

(102277, 19)